# comercio_ext_indices.tb_referencia_ncm_ppe
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/NCM_PPE_delta`
## 📌 Descrição do arquivo

Classificação **PPE** usada para categorizar produtos, com descrições em PT/EN.

|Coluna|Descrição técnica|Interpretação humana|
|---|---|---|
|`CO_PPE`|Código PPE|Identificador PPE|
|`NO_PPE`|Nome (PT)|Descrição do PPE em português|
|`NO_PPE_MIN`|Nome (PT – minúsculo)|Variante textual|
|`NO_PPE_ING`|Nome (EN)|Descrição em inglês|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from delta.tables import DeltaTable
jvm = spark._jvm
FileSystem = jvm.org.apache.hadoop.fs.FileSystem
Path = jvm.org.apache.hadoop.fs.Path
fs = FileSystem.get(spark._jsc.hadoopConfiguration())

In [0]:
bronzePath = "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/ncm_ppe"
silverPath = "/mnt/silver/landingbeca2026jan/comercio_ext_indices/tb_referencia_ncm_ppe/"
silverTable = "silver_comercio_ext_indices.tb_referencia_ncm_ppe"

In [0]:
silverSchema = T.StructType([
    T.StructField("CO_PPE",     T.StringType(),   False),
    T.StructField("NO_PPE",     T.StringType(),   False),
    T.StructField("NO_PPE_MIN", T.StringType(),   True ),
    T.StructField("NO_PPE_ING", T.StringType(),   True ),
    T.StructField("TS_REF",     T.TimestampType(),False),
    T.StructField("NM_ORIGEM",  T.StringType(),   False),
])

## Extração
> #### **saprk.read**

In [0]:
df_bronze = spark.read.format("delta").load(bronzePath)

##Normalização
> #### **datatype**

In [0]:
def up(s): return F.upper(F.trim(F.col(s).cast(T.StringType())))

df_norm = (
    df_bronze
    .withColumn("CO_PPE", up("CO_PPE"))
    .withColumn("NO_PPE", F.col("NO_PPE").cast(T.StringType()))
    .withColumn("NO_PPE_MIN", F.col("NO_PPE_MIN").cast(T.StringType()))
    .withColumn("NO_PPE_ING", F.col("NO_PPE_ING").cast(T.StringType()))
    .withColumn("TS_REF", F.current_timestamp())
    .withColumn("NM_ORIGEM", F.lit("/landingbeca2026jan/balancacomercial/ncm_ppe"))
)

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:
df_valid = df_norm.filter(F.col("CO_PPE").isNotNull())


In [0]:

df_dedup = df_valid.dropDuplicates(["CO_PPE"])

In [0]:
df_silver = df_dedup.select("CO_PPE","NO_PPE","NO_PPE_MIN","NO_PPE_ING","TS_REF","NM_ORIGEM")

##Carga
> #### **merge**

In [0]:
delta_target = DeltaTable.forName(spark, "silver_comercio_ext_indices.tb_referencia_ncm_ppe")

merge_condition = """
  t.CO_PPE = s.CO_PPE
"""

(delta_target.alias("t")
    .merge(df_silver.alias("s"), merge_condition)
    .whenMatchedUpdate(set={
        "t.NO_PPE": "s.NO_PPE",
        "t.NO_PPE_MIN": "s.NO_PPE_MIN",
        "t.NO_PPE_ING": "s.NO_PPE_ING",
        "t.TS_REF": "s.TS_REF",
        "t.NM_ORIGEM": "s.NM_ORIGEM",
    })
    .whenNotMatchedInsertAll()
    .execute()
)